In [ ]:
#!pip install google-cloud-bigquery pandas db-dtypes google-cloud-bigquery-storage
import os
from google.cloud import bigquery
import pandas as pd
import pickle
from datetime import timedelta

import db_dtypes


print("db-dtypes is installed correctly.")


# Set the credentials for Google BigQuery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "st312-442610-b9934d6c7287.json"
client = bigquery.Client()


db-dtypes is installed correctly.


In [ ]:
matched_df = pd.read_csv("matched_orders_sample.csv")
filled_df = pd.read_csv("filled_orders_sample.csv")


filled_df

In [153]:
## step 1 class Person
from datetime import timedelta



class People:
    def __init__(self):
        self.people = {}

    def add_person(self, unique_id, balance):
        self.people[unique_id] = Person(unique_id, balance)
    
    
    def potential_add_person(self, unique_id):

        if unique_id not in self.people:
            self.add_person(unique_id, 0)

    def plot_returns(self, exchange):
        """
        Plots the returns for each person in the exchange.
        
        :param exchange: StockExchange object.
        """

        returns = []
        for person in self.people.values():
            returns.append(person.get_total_return(exchange))
        

        plt.hist(returns, bins=20, weights=[100/len(returns)]*len(returns), edgecolor='black')


        plt.title('Histogram of Returns')
        plt.xlabel('Value')
        plt.ylabel('Frequency (%)')

        # Show the plot
        plt.show()

    def make_df(self, exchange):
        """
        Creates a DataFrame from the people data.
        
        :return: DataFrame with columns ['id', 'balance', 'total_paid', 'total_made', 'portfolio', 'order_history'].
        """
        data = []
        for person in self.people.values():
            data.append([person.id, person.get_total_return(exchange), person.calculate_momentum_score(exchange), person.calculate_average_holding_time(), person.calculate_risk_tolerance(), person.calculate_intraday_closeout_rate(), person.calculate_order_clustering_score(), person.calculate_asset_diversification_index(), person.calculate_stop_loss_index(exchange)])
        return pd.DataFrame(data, columns=['id', 'return', 'momentum_score', 'average_holding_time', 'risk_tolerance', 'intraday_closeout_rate', 'order_clustering_score', 'asset_diversification_index', 'stop_loss_index'])


from datetime import datetime

class Person:
    def __init__(self, unique_id, balance):
        self.id = unique_id
        self.balance = balance
        self.portfolio = {}  # {stock_symbol: {"shares": total_shares, "total_cost": total_cost}}
        self.total_paid = 0
        self.total_made = 0
        self.order_history = []


    def buy(self, stock, quantity, total_cost, timeStamp):
        
        self.balance -= total_cost
        
        if stock not in self.portfolio:
            self.portfolio[stock] = {"shares": 0, "total_cost": 0}
        
        self.portfolio[stock]["shares"] += quantity
        self.portfolio[stock]["total_cost"] += total_cost

        self.total_paid += total_cost
        self.order_history.append({'action':"buy", "stock":stock ,"quant":quantity, "cost":total_cost, "time":timeStamp})

    
    def sell(self, stock, quantity, total_revenue, timeStamp):
        if stock not in self.portfolio or self.portfolio[stock]["shares"] < quantity:
            #print("Ignoring as bought before the study period")
            #raise ValueError("Insufficient shares to sell.")

            pass

        else:
            
            # Calculate the average price per share
            average_price = self.portfolio[stock]["total_cost"] / self.portfolio[stock]["shares"]
            
            # Reduce the number of shares and adjust the total cost proportionally
            self.portfolio[stock]["shares"] -= quantity
            self.portfolio[stock]["total_cost"] -= average_price * quantity
            

            # Add the revenue to the balance
            self.balance += total_revenue

            self.total_made += total_revenue

            self.order_history.append({'action':"sell", "stock":stock, "quant":quantity, "cost":total_revenue, "time":timeStamp})
            
            # Remove the stock entry if all shares are sold
            if self.portfolio[stock]["shares"] == 0:
                del self.portfolio[stock]
            
            

    
    def get_portfolio_value(self, exchange):
        total_value = 0
        for stock, data in self.portfolio.items():
            stock_value = data["shares"] * exchange.stocks[stock].get_latest_price()[1]
            total_value += stock_value
        return total_value
    
    def get_average_price(self, stock):
        if stock in self.portfolio and self.portfolio[stock]["shares"] > 0:
            return self.portfolio[stock]["total_cost"] / self.portfolio[stock]["shares"]
        return 0
    
    def get_total_return(self, exchange):
        if self.total_paid == 0:
            return 0
        total_income = self.total_made + self.get_portfolio_value(exchange)
        total_expense = self.total_paid
        return (total_income - total_expense) / total_expense
    
    def calculate_average_holding_time(self):
        """
        Calculates the average holding time across all stocks that were both bought and sold.
        Returns the average time in days.
        """
        from dateutil import parser
        
        sorted_orders = sorted(self.order_history, key=lambda x: x["time"])
        buy_times = {}  # Store buy timestamps for each stock
        holding_seconds = []  # Store holding times in seconds

        for order in sorted_orders:
            stock = order["stock"]
            
            # Ensure timestamp is a datetime object
            timestamp = order["time"]
            if isinstance(timestamp, str):
                try:
                    timestamp = parser.parse(timestamp)
                except:
                    continue
            
            if order["action"] == "buy":
                if stock not in buy_times:
                    buy_times[stock] = timestamp
            elif order["action"] == "sell" and stock in buy_times:
                try:
                    holding_time = (timestamp - buy_times[stock]).total_seconds()
                    holding_seconds.append(holding_time)
                    del buy_times[stock]
                except Exception:
                    continue

        # Calculate average in days
        if not holding_seconds:
            return 0
        
        avg_seconds = sum(holding_seconds) / len(holding_seconds)
        avg_days = avg_seconds / (60 * 60 * 24)  # Convert seconds to days
        
        return avg_days


    
    def calculate_momentum_score(self, exchange, lookback_period=5):
        """
        Calculates a momentum score for the person based on their order history.
        
        A positive score indicates momentum trading (buying rising stocks, selling falling ones).
        A negative score indicates contrarian trading (buying falling stocks, selling rising ones).
        
        :param exchange: StockPortfolio object containing historical prices
        :param lookback_period: Number of price points to look back to determine trend
        :return: Momentum score (float)
        """
        momentum_score = 0
        trade_count = 0
        
        for order in self.order_history:
            stock = order["stock"]
            action = order["action"]
            timestamp = order["time"]
            
            # Skip if stock doesn't exist in exchange
            if stock not in exchange.stocks:
                continue
                
            # Get stock price history before this order
            stock_prices = exchange.stocks[stock].get_prices()
            
            # Find prices before the current timestamp
            prior_prices = [(t, p) for t, p in stock_prices if t < timestamp]
            
            if len(prior_prices) >= lookback_period:
                # Get the most recent prices before the order
                recent_prices = prior_prices[-lookback_period:]
                
                # Calculate simple trend: comparing last price to first price in the window
                first_price = recent_prices[0][1]
                last_price = recent_prices[-1][1]
                
                is_uptrend = last_price > first_price
                
                # Score based on action and trend
                if action == "buy" and is_uptrend:
                    momentum_score += 1  # Buying in uptrend (momentum behavior)
                elif action == "buy" and not is_uptrend:
                    momentum_score -= 1  # Buying in downtrend (contrarian behavior)
                elif action == "sell" and is_uptrend:
                    momentum_score -= 1  # Selling in uptrend (contrarian behavior)
                elif action == "sell" and not is_uptrend:
                    momentum_score += 1  # Selling in downtrend (momentum behavior)
                
                trade_count += 1
        
        # Normalize by number of valid trades
        if trade_count > 0:
            momentum_score /= trade_count
        
        return momentum_score
    
    def calculate_risk_tolerance(self):
        """
        Calculates a risk tolerance score based on the prices of assets purchased.
        
        In Polymarket context:
        - Low-priced assets (near 0) are considered high risk (long shots)
        - High-priced assets (near 1) are considered low risk (sure bets)
        
        Returns a score from 0 to 1:
        - 0: Extremely risk-averse (only buys high-priced assets)
        - 1: Extremely risk-tolerant (only buys low-priced assets)
        """
        buy_orders = [order for order in self.order_history if order['action'] == 'buy']
        
        if not buy_orders:
            return 0.5  # Neutral score if no buy orders
        
        weighted_risk_score = 0
        total_spent = 0
        
        for order in buy_orders:
            quantity = order['quant']
            total_cost = order['cost']
            
            # Calculate price per share (0 to 1 in Polymarket)
            if quantity > 0:
                price_per_share = total_cost / quantity
                
                # The risk score for this transaction - inverse of price
                # (lower price = higher risk = higher score)
                risk_score = 1 - price_per_share
                
                # Weight by amount spent
                weighted_risk_score += risk_score * total_cost
                total_spent += total_cost
        
        # Calculate weighted average
        if total_spent > 0:
            avg_risk_tolerance = weighted_risk_score / total_spent
            return avg_risk_tolerance
        
        return 0.5  # Default neutral score

    def calculate_intraday_closeout_rate(self):
        """
        Calculates the percentage of trading days where positions opened
        were closed within the same day.
        
        Returns a value from 0 to 1:
        - 0: Never closes positions same day
        - 1: Always closes all positions same day (characteristic of HFT)
        """
        from dateutil import parser
        from collections import defaultdict
        
        # Group orders by day and stock
        daily_positions = defaultdict(lambda: defaultdict(lambda: {'buys': 0, 'sells': 0}))
        
        for order in self.order_history:
            timestamp = order['time']
            if isinstance(timestamp, str):
                try:
                    timestamp = parser.parse(timestamp)
                except:
                    continue
            
            day = timestamp.date()
            stock = order['stock']
            quantity = order['quant']
            
            if order['action'] == 'buy':
                daily_positions[day][stock]['buys'] += quantity
            elif order['action'] == 'sell':
                daily_positions[day][stock]['sells'] += quantity
        
        # Calculate closeout rate
        days_with_trades = 0
        days_with_full_closeout = 0
        
        for day, stocks in daily_positions.items():
            if not stocks:  # Skip days with no trading
                continue
                
            days_with_trades += 1
            
            # Check if all positions opened this day were closed by end of day
            day_closed = True
            for stock_data in stocks.values():
                if stock_data['buys'] > stock_data['sells']:
                    day_closed = False
                    break
                    
            if day_closed:
                days_with_full_closeout += 1
        
        # Return the closeout rate
        if days_with_trades == 0:
            return 0
        return days_with_full_closeout / days_with_trades

    def calculate_order_clustering_score(self, time_window_seconds=300):
        """
        Calculates how clustered a trader's orders are in time.
        
        High-frequency traders typically place orders in rapid bursts.
        
        Parameters:
        - time_window_seconds: Window to consider for clustering analysis (default: 5 minutes)
        
        Returns a score from 0 to 1:
        - 0: Orders evenly spaced out (low clustering)
        - 1: Orders highly clustered in time (characteristic of HFT)
        """
        from dateutil import parser
        from collections import defaultdict
        import numpy as np
        
        # Extract timestamps and convert to datetime
        timestamps = []
        for order in self.order_history:
            time = order['time']
            if isinstance(time, str):
                try:
                    time = parser.parse(time)
                    timestamps.append(time)
                except:
                    continue
            else:
                timestamps.append(time)
        
        if len(timestamps) <= 1:
            return 0  # Not enough data
        
        # Sort timestamps
        timestamps.sort()
        
        # Calculate time gaps between consecutive orders in seconds
        time_gaps = [(timestamps[i+1] - timestamps[i]).total_seconds() 
                    for i in range(len(timestamps)-1)]
        
        if not time_gaps:
            return 0
        
        # Calculate clustering based on proportion of gaps smaller than the window
        small_gaps = sum(1 for gap in time_gaps if gap < time_window_seconds)
        clustering_score = small_gaps / len(time_gaps)
        
        # Alternative approach: Calculate burst intensity
        # Count orders within sliding windows
        burst_intensity = 0
        for i in range(len(timestamps)):
            window_end = timestamps[i]
            window_start = window_end - timedelta(seconds=time_window_seconds)
            orders_in_window = sum(1 for t in timestamps if window_start <= t <= window_end)
            burst_intensity = max(burst_intensity, orders_in_window)
        
        # Normalize burst intensity (cap at 20 orders per window for normalization)
        normalized_burst = min(burst_intensity / 20, 1.0)
        
        # Combine both metrics (can adjust weights as needed)
        combined_score = 0.7 * clustering_score + 0.3 * normalized_burst
        
        return combined_score
    
    def calculate_asset_diversification_index(self):
        """
        Calculates how diversified a person's investments are across different assets.
        
        Returns:
        - unique_assets_count: Number of unique assets traded
        - concentration_ratio: Ratio of most traded asset volume to total trading volume
        - diversity_score: Score from 0-1 where 1 is perfectly diversified across assets
        """
        if not self.order_history:
            return 0, 1.0, 0
            
        # Count unique assets
        unique_assets = set(order['stock'] for order in self.order_history)
        unique_assets_count = len(unique_assets)
        
        # Calculate volume by asset
        asset_volumes = {}
        total_volume = 0
        
        for order in self.order_history:
            stock = order['stock']
            quantity = order['quant']
            
            if stock not in asset_volumes:
                asset_volumes[stock] = 0
            asset_volumes[stock] += quantity
            total_volume += quantity
        
        # Calculate concentration ratio (how much is in the most traded asset)
        max_asset_volume = max(asset_volumes.values()) if asset_volumes else 0
        concentration_ratio = max_asset_volume / total_volume if total_volume > 0 else 1.0
        
        # Calculate diversity score using Herfindahl-Hirschman Index approach
        # (adapted from portfolio theory)
        if total_volume > 0:
            hhi = sum((vol/total_volume)**2 for vol in asset_volumes.values())
            # Convert to a 0-1 score where 1 is perfectly diversified
            diversity_score = 1 - (hhi - 1/len(asset_volumes))/(1 - 1/len(asset_volumes)) if len(asset_volumes) > 1 else 0
        else:
            diversity_score = 0
            
        return unique_assets_count, concentration_ratio, diversity_score
    
    def calculate_stop_loss_index(self, exchange):
        """
        Calculates metrics related to stop loss behavior.
        
        Returns:
        - stop_loss_frequency: Percentage of sells that appear to be stop losses
        - stop_loss_threshold: Average percentage drop at which stop losses are triggered  
        - stop_loss_discipline_index: A composite score from 0-1 measuring stop loss usage
        """
        from dateutil import parser
        import numpy as np
        
        # Track pairs of buys and sells for the same asset
        buys_by_stock = {}
        for order in self.order_history:
            if order['action'] == 'buy':
                stock = order['stock']
                timestamp = order['time']
                if isinstance(timestamp, str):
                    try:
                        timestamp = parser.parse(timestamp)
                    except:
                        continue
                
                quantity = order['quant']
                cost = order['cost']
                price_per_share = cost / quantity if quantity > 0 else 0
                
                if stock not in buys_by_stock:
                    buys_by_stock[stock] = []
                
                buys_by_stock[stock].append({
                    'timestamp': timestamp,
                    'quantity': quantity,
                    'price': price_per_share
                })
        
        # For each sell, find the corresponding buy and calculate price change
        price_changes = []
        for order in self.order_history:
            if order['action'] == 'sell':
                stock = order['stock']
                timestamp = order['time']
                if isinstance(timestamp, str):
                    try:
                        timestamp = parser.parse(timestamp)
                    except:
                        continue
                
                # Skip if no previous buys for this stock
                if stock not in buys_by_stock or not buys_by_stock[stock]:
                    continue
                
                # Calculate sell price
                quantity = order['quant']
                revenue = order['cost']  # In your data 'cost' field is revenue for sells
                sell_price = revenue / quantity if quantity > 0 else 0
                
                # Find the most recent buy before this sell
                valid_buys = [buy for buy in buys_by_stock[stock] if buy['timestamp'] < timestamp]
                if not valid_buys:
                    continue
                
                # Get the most recent buy
                valid_buys.sort(key=lambda x: x['timestamp'])
                most_recent_buy = valid_buys[-1]
                
                # Calculate price change percentage
                buy_price = most_recent_buy['price']
                price_change_pct = (sell_price - buy_price) / buy_price if buy_price > 0 else 0
                
                # Record the price change
                price_changes.append(price_change_pct)
                
                # Update the buy quantity
                most_recent_buy['quantity'] -= quantity
                if most_recent_buy['quantity'] <= 0:
                    buys_by_stock[stock].remove(most_recent_buy)
        
        # If no price changes, return default values
        if not price_changes:
            return 0, 0, 0
        
        # Calculate metrics
        # 1. Stop Loss Frequency - percentage of sells that occurred at a loss
        losses = [change for change in price_changes if change < 0]
        stop_loss_frequency = len(losses) / len(price_changes) if price_changes else 0
        
        # 2. Stop Loss Threshold - average percentage drop at which stops are triggered
        stop_loss_threshold = abs(np.mean(losses)) if losses else 0
        
        # 3. Stop Loss Consistency - coefficient of variation of loss percentages
        loss_std = np.std(losses) if losses and len(losses) > 1 else 0
        loss_mean = abs(np.mean(losses)) if losses else 1
        stop_loss_consistency = 1 - min(loss_std / loss_mean if loss_mean > 0 else 0, 1)
        
        # 4. Calculate discipline index based on consistency and frequency
        if stop_loss_frequency > 0 and stop_loss_threshold > 0:
            # Higher score for traders who consistently use stop losses at similar levels
            stop_loss_discipline_index = (stop_loss_frequency * 0.4) + (stop_loss_consistency * 0.6)
        else:
            # Low score for traders who rarely use stop losses
            stop_loss_discipline_index = 0
        
        return stop_loss_frequency, stop_loss_threshold, stop_loss_discipline_index



    





import matplotlib.pyplot as plt
from datetime import datetime

class StockPrices:
    def __init__(self, symbol):
        self.symbol = symbol
        self.prices = []  # List of tuples: [(timestamp, price), ...]
    
    def add_price(self, timestamp, price):
        """
        Adds a new price with a timestamp.
        
        :param timestamp: Datetime object representing the time of the sale.
        :param price: Sale price of the stock.
        """

        self.prices.append((timestamp, price))
        self.prices.sort(key=lambda x: x[0], reverse=False)

    
    def get_prices(self):
        """
        Returns the list of recorded prices.
        
        :return: List of tuples [(timestamp, price), ...].
        """
        return self.prices
    
    def plot_prices(self):
        """
        Plots the evolution of stock prices over time.
        """
        if not self.prices:
            print("No prices to plot.")
            return
        
        # Extract timestamps and prices
        timestamps, prices = zip(*self.prices)
        
        # Plot prices
        plt.figure(figsize=(10, 5))
        plt.plot(timestamps, prices, marker="o", linestyle="-")
        plt.title(f"Price Evolution for {self.symbol}")
        plt.xlabel("Timestamp")
        plt.ylabel("Price")
        plt.grid(True)
        plt.show()
    
    def get_latest_price(self):
        """
        Retrieves the latest price based on timestamp.
        
        :return: Latest price or None if no prices exist.
        """
        if not self.prices:
            return None
        return self.prices[-1]
    

## AllStocks class

class StockPortfolio:
    def __init__(self):
        self.stocks = {}  # Maps stock symbols to StockPrices objects
    
    def add_stock(self, symbol):
        if symbol not in self.stocks:
            self.stocks[symbol] = StockPrices(symbol)
    
    def add_price(self, symbol, timestamp, price):
        if symbol not in self.stocks:
            self.add_stock(symbol)
        self.stocks[symbol].add_price(timestamp, price)
    
    def get_stock(self, symbol):
        return self.stocks.get(symbol, None)
    
    def plot_all_prices(self):
        for stock in self.stocks.values():
            stock.plot_prices()


In [154]:


people = People()
stockExchange = StockPortfolio()


def handle_orders(df):




    maker = df.iloc[0]['maker']
    taker = df.iloc[0]['taker']
    maker_asset_id = df.iloc[0]['maker_asset_id']
    taker_asset_id = df.iloc[0]['taker_asset_id']
    maker_amount = df.iloc[0]['maker_amount']
    taker_amount = df.iloc[0]['taker_amount']

    timeStamp = df.iloc[0]['block_timestamp']

    
    

    people.potential_add_person(maker)
    people.potential_add_person(taker)



    if maker_asset_id == "0000000000000000000000000000000000000000000000000000000000000000":
        ### Maker is buying!

        people.people[maker].buy(taker_asset_id, taker_amount, maker_amount, timeStamp) 
        people.people[taker].sell(taker_asset_id, taker_amount, maker_amount, timeStamp)

        stockExchange.add_price(taker_asset_id, timeStamp, maker_amount/taker_amount)

    if taker_asset_id == "0000000000000000000000000000000000000000000000000000000000000000":
        # Taker is selling!

        people.people[taker].buy(maker_asset_id,maker_amount, taker_amount, timeStamp)
        people.people[maker].sell(maker_asset_id, maker_amount, taker_amount, timeStamp) 
        stockExchange.add_price(maker_asset_id, timeStamp, taker_amount/maker_amount)

    


import ast



for index, row in matched_df.iterrows():



    topics_list = ast.literal_eval(row["topics"])


    taker_order_hash = topics_list[0][66:132]







    transaction_hash = row["transaction_hash"]
    
    data = row["data"]
    maker_asset_id = data[2:66]
    taker_asste_id = data[66:130]
    maker_amount = int(data[130:194], 16)
    taker_amount = int(data[194:258], 16)


    ### Now i get into order fillin!

    cur_filled = filled_df[filled_df['transaction_hash'] == transaction_hash].copy()
    






    cur_filled.loc[:, "takeOrderHash"] = cur_filled["topics"].apply(lambda x: ast.literal_eval(x)[0][66:132]) 
                                                                    
                                                                 




    cur_filled = cur_filled[cur_filled["takeOrderHash"] == taker_order_hash]

    if len(cur_filled) == 0:
        print("only 0")
    elif len(cur_filled) == 1:

        


        cur_filled.loc[:, "maker"] = cur_filled["topics"].apply(lambda x: ast.literal_eval(x)[0][132:198])


        cur_filled.loc[:, "taker"] = cur_filled["topics"].apply(lambda x: ast.literal_eval(x)[0][198:272])

        cur_filled.loc[:, "maker_asset_id"] = cur_filled["data"].str[2:66]
        cur_filled.loc[:, "taker_asset_id"] = cur_filled["data"].str[66:130]
  
        cur_filled.loc[:, "maker_amount"] = cur_filled["data"].str[130:194].apply(lambda x: int(x, 16))
        cur_filled.loc[:, "taker_amount"] = cur_filled["data"].str[194:258].apply(lambda x: int(x, 16))



        subset_df = cur_filled[["maker", "taker", "maker_asset_id", "taker_asset_id", "maker_amount", "taker_amount", "block_timestamp"]]


        
        handle_orders(subset_df)
    else:
        display(cur_filled)
        print(transaction_hash)




In [155]:
people.make_df(stockExchange)

,id,return,momentum_score,average_holding_time,risk_tolerance,intraday_closeout_rate,order_clustering_score,asset_diversification_index,stop_loss_index
0,0x0000000000000000000000005e1f859c104cfc94b0fa...,0.000000,0.000000,0.000000,0.500000,0.0,0.000000,"(0, 1.0, 0)","(0, 0, 0)"
1,0x000000000000000000000000c5d563a36ae78145c45a...,0.055050,-0.072280,0.822029,0.275287,0.0,0.982955,"(379, 0.11281716618748922, 0.9514763219233953)","(0.08619173262972735, 0.09071850297611399, 0.0..."
2,0x00000000000000000000000059ee6c6a56d7b00223f0...,-0.001452,-0.333333,0.000000,0.006999,0.0,0.164512,"(23, 0.8342368056392336, 0.3148764111477895)","(0, 0, 0)"
3,0x000000000000000000000000e0b7e9c90bf84274a846...,0.002008,0.000000,0.000000,0.004000,0.0,0.000000,"(1, 1.0, 0)","(0, 0, 0)"
4,0x000000000000000000000000c580c51f81c033d7c300...,-0.018266,-1.000000,0.000000,0.010951,0.0,0.015000,"(5, 0.31970260223048325, 0.8934370724561573)","(0, 0, 0)"
...,...,...,...,...,...,...,...,...,...
1872,0x000000000000000000000000bf2c878cd5d6043697b3...,0.005682,1.000000,0.000000,0.296000,0.0,0.000000,"(1, 1.0, 0)","(0, 0, 0)"
1873,0x0000000000000000000000004eb3cebba2ca0d104c35...,0.000000,1.000000,0.000000,0.995000,0.0,0.000000,"(1, 1.0, 0)","(0, 0, 0)"
1874,0x000000000000000000000000d65529d589c569b139e0...,0.000000,0.000000,0.000000,0.997000,0.0,0.000000,"(1, 1.0, 0)","(0, 0, 0)"
1875,0x00000000000000000000000087a8cc291cb6a170ad4c...,0.001002,-1.000000,0.000000,0.002000,0.0,0.000000,"(1, 1.0, 0)","(0, 0, 0)"


In [27]:
cur_filled

,transaction_hash,address,data,topics,block_timestamp,takeOrderHash
11150,0x84f158638a1bbd5f641b6e6d5c52104dc4b34677fba5...,0xc5d563a36ae78145c45a50134d48a1215220f80a,0x36d38d5b2a2110f83cd39b91595c95c26fc8b10355a2...,['0xd0a08e8c493f9c94f29311604c9de1b4e8c8d4c06b...,2024-07-22 17:22:04+00:00,'
11151,0x84f158638a1bbd5f641b6e6d5c52104dc4b34677fba5...,0xc5d563a36ae78145c45a50134d48a1215220f80a,0x00000000000000000000000000000000000000000000...,['0xd0a08e8c493f9c94f29311604c9de1b4e8c8d4c06b...,2024-07-22 17:22:04+00:00,'
11153,0x84f158638a1bbd5f641b6e6d5c52104dc4b34677fba5...,0xc5d563a36ae78145c45a50134d48a1215220f80a,0x00000000000000000000000000000000000000000000...,['0xd0a08e8c493f9c94f29311604c9de1b4e8c8d4c06b...,2024-07-22 17:22:04+00:00,'


,id,return
0,0x0000000000000000000000005e1f859c104cfc94b0fa...,0.000000
1,0x000000000000000000000000c5d563a36ae78145c45a...,0.055050
2,0x00000000000000000000000059ee6c6a56d7b00223f0...,-0.001452
3,0x000000000000000000000000e0b7e9c90bf84274a846...,0.002008
4,0x000000000000000000000000c580c51f81c033d7c300...,-0.018266
...,...,...
1872,0x000000000000000000000000bf2c878cd5d6043697b3...,0.005682
1873,0x0000000000000000000000004eb3cebba2ca0d104c35...,0.000000
1874,0x000000000000000000000000d65529d589c569b139e0...,0.000000
1875,0x00000000000000000000000087a8cc291cb6a170ad4c...,0.001002


In [ ]:
 for person in self.people.values():
            data.append([person.id, person.balance, person.total_paid, person.total_made, person.portfolio, person.order_history])
        return pd.DataFrame(data, columns=['id', 'balance', 'total_paid', 'total_made', 'portfolio', 'order_history'])